# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [22]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [23]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [24]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [25]:


def flow_with_demands(graph):
    import networkx as nx
    from networkx.algorithms.flow import maximum_flow
    # reduce it to maxflow
    for node in graph.nodes():
        if graph.node[node]['demand'] < 0:
            graph.add_edge('source',node)
            graph.edge['source'][node]['capacity']= abs(graph.node[node]['demand'])
        
        elif graph.node[node]['demand'] > 0:
            graph.add_edge(node,'sink')
            graph.edge[node]['sink']['capacity']= abs(graph.node[node]['demand'])
    
    demandcheck=0
    for node in graph.nodes():
        demandcheck+=graph.node[node]['demand']
        
    # to check if minCut satisfies demand later
    totalsourcecapacity = 0
    for node in graph.nodes():
        try:
            totalsourcecapacity += graph.edge['source'][node]['capacity']
        except KeyError:
            continue
    
    #max flow
    (flow1,flow2)= nx.maximum_flow(graph,'source','sink')
    
    
    #retract reduction
    for node in graph.nodes():
        try:
            del flow2[node]['sink']
            del flow2['source'][node]
        except KeyError:
            continue
    del flow2['source']
    del flow2['sink']
        
    graph.remove_node('source')
    graph.remove_node('sink')
    

    
    #exceptions 
    #these could be better placed for a more efficient algorithm but decided to keep it legible
    
    
    if demandcheck != 0:
        raise nx.NetworkXUnfeasible('Not possible: Demand is not symmetric ')
    
    if flow1 == totalsourcecapacity:
        return flow2

    else:
        raise nx.NetworkXUnfeasible('No flow satisfying the demand')
        

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [26]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict."""
    #create dict
    flowdict={}
    for key in flow:
        flowdict[key]=0
        for subkey in flow[key]:
            flowdict[subkey]=0
            
    #flow out
    for key in flow:
        for subkey in flow[key]:
            flowdict[key] -= flow[key][subkey]

            
    #flow in
    for key in flow:
        for subkey in flow[key]:
            try:
                flowdict[subkey] += flow[key][subkey]
            except KeyError:
                continue
            

    return flowdict

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [27]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
